<a href="https://colab.research.google.com/github/dzejkopjarosinski/ZespolR/blob/main/MLF_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [172]:
#Modelowanie
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
from scipy.optimize import minimize
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.linear_model import LinearRegression
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
import yfinance as yf

# Dane i Target Variable

In [173]:
df = pd.read_csv("/content/DataFrame_SP500_all.csv")

In [174]:
df = df.drop('Unnamed: 0',axis=1)

In [175]:
df.columns

Index(['ticker', 'date', 'asset_debt_coverage', 'net_income',
       'return_on_equity', 'return_on_assets', 'current_assets',
       'current_liabilities', 'current_ratio', 'total_asset',
       'total_liabilities', 'debt_to_equity_ratio', 'debt_to_assets_ratio'],
      dtype='object')

In [176]:
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
df.head()

,ticker,date,asset_debt_coverage,net_income,return_on_equity,return_on_assets,current_assets,current_liabilities,current_ratio,total_asset,total_liabilities,debt_to_equity_ratio,debt_to_assets_ratio
0,MMM,2019-12-31,2.172130,5.027400e+10,4.686976e+09,1.125731,1.127800e+10,9.222000e+09,1.222945,4.465900e+10,3.453300e+10,3.410330,0.773260
1,MMM,2020-12-31,2.485119,5.142200e+10,3.754085e+09,1.086136,1.357800e+10,7.948000e+09,1.708354,4.734400e+10,3.441300e+10,2.661279,0.726871
2,MMM,2021-12-31,2.670600,5.257100e+10,3.282977e+09,1.116821,1.420900e+10,9.035000e+09,1.572662,4.707200e+10,3.195500e+10,2.113845,0.678854
3,MMM,2022-12-31,2.867593,5.462300e+10,3.491261e+09,1.175826,1.355900e+10,9.523000e+09,1.423816,4.645500e+10,3.168500e+10,2.145227,0.682058
4,AOS,2022-12-31,9.082311,2.967400e+09,3.235789e+08,0.890496,1.488800e+09,9.342000e+08,1.593663,3.332300e+09,1.584600e+09,0.906677,0.475527


In [177]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1992 entries, 0 to 1991
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   ticker                1992 non-null   object        
 1   date                  1992 non-null   datetime64[ns]
 2   asset_debt_coverage   1973 non-null   float64       
 3   net_income            1937 non-null   float64       
 4   return_on_equity      1936 non-null   float64       
 5   return_on_assets      1937 non-null   float64       
 6   current_assets        1354 non-null   float64       
 7   current_liabilities   1800 non-null   float64       
 8   current_ratio         1353 non-null   float64       
 9   total_asset           1992 non-null   float64       
 10  total_liabilities     1992 non-null   float64       
 11  debt_to_equity_ratio  1991 non-null   float64       
 12  debt_to_assets_ratio  1992 non-null   float64       
dtypes: datetime64[ns](

In [178]:
tickers = ["MMM","AOS","ABT","ABBV","ACN","ATVI","ADM","ADBE","ADP","AAP","AES","AFL",
           "A","APD","AKAM","ALK","ALB","ARE","ALGN","ALLE","LNT","ALL","GOOGL","GOOG",
           "MO","AMZN","AMCR","AMD","AEE","AAL","AEP","AXP","AIG","AMT","AWK","AMP",
           "ABC","AME","AMGN","APH","ADI","ANSS","AON","APA","AAPL","AMAT","APTV",
           "ACGL","ANET","AJG","AIZ","T","ATO","ADSK","AZO","AVB","AVY","AXON","BKR","BALL",
           "BAC","BBWI","BAX","BDX","WRB","BRK.B","BBY","BIO","TECH","BIIB","BLK","BK","BA",
           "BKNG","BWA","BXP","BSX","BMY","AVGO","BR","BRO","BF.B","BG","CHRW","CDNS","CZR",
           "CPT","CPB","COF","CAH","KMX","CCL","CARR","CTLT","CAT","CBOE","CBRE","CDW","CE",
           "CNC","CNP","CDAY","CF","CRL","SCHW","CHTR","CVX","CMG","CB","CHD","CI","CINF",
           "CTAS","CSCO","C","CFG","CLX","CME","CMS","KO","CTSH","CL","CMCSA","CMA","CAG",
           "COP","ED","STZ","CEG","COO","CPRT","GLW","CTVA","CSGP","COST","CTRA","CCI",
           "CSX","CMI","CVS","DHI","DHR","DRI","DVA","DE","DAL","XRAY","DVN","DXCM","FANG",
           "DLR","DFS","DISH","DIS","DG","DLTR","D","DPZ","DOV","DOW","DTE","DUK","DD","DXC",
           "EMN","ETN","EBAY","ECL","EIX","EW","EA","ELV","LLY","EMR","ENPH","ETR","EOG",
           "EPAM","EQT","EFX","EQIX","EQR","ESS","EL","ETSY","RE","EVRG","ES","EXC","EXPE",
           "EXPD","EXR","XOM","FFIV","FDS","FICO","FAST","FRT","FDX","FITB","FSLR","FE",
           "FIS","FISV","FLT","FMC","F","FTNT","FTV","FOXA","FOX","BEN","FCX","GRMN","IT",
           "GEHC","GEN","GNRC","GD","GE","GIS","GM","GPC","GILD","GL","GPN","GS","HAL","HIG",
           "HAS","HCA","PEAK","HSIC","HSY","HES","HPE","HLT","HOLX","HD","HON","HRL","HST",
           "HWM","HPQ","HUM","HBAN","HII","IBM","IEX","IDXX","ITW","ILMN","INCY","IR","PODD",
           "INTC","ICE","IFF","IP","IPG","INTU","ISRG","IVZ","INVH","IQV","IRM","JBHT","JKHY",
           "J","JNJ","JCI","JPM","JNPR","K","KDP","KEY","KEYS","KMB","KIM","KMI","KLAC","KHC",
           "KR","LHX","LH","LRCX","LW","LVS","LDOS","LEN","LNC","LIN","LYV","LKQ","LMT","L",
           "LOW","LYB","MTB","MRO","MPC","MKTX","MAR","MMC","MLM","MAS","MA","MTCH","MKC","MCD",
           "MCK","MDT","MRK","META","MET","MTD","MGM","MCHP","MU","MSFT","MAA","MRNA","MHK","MOH",
           "TAP","MDLZ","MPWR","MNST","MCO","MS","MOS","MSI","MSCI","NDAQ","NTAP","NFLX","NWL","NEM",
           "NWSA","NWS","NEE","NKE","NI","NDSN","NSC","NTRS","NOC","NCLH","NRG","NUE","NVDA","NVR",
           "NXPI","ORLY","OXY","ODFL","OMC","ON","OKE","ORCL","OGN","OTIS","PCAR","PKG","PARA","PH",
           "PAYX","PAYC","PYPL","PNR","PEP","PKI","PFE","PCG","PM","PSX","PNW","PXD","PNC","POOL",
           "PPG","PPL","PFG","PG","PGR","PLD","PRU","PEG","PTC","PSA","PHM","QRVO","PWR","QCOM","DGX",
           "RL","RJF","RTX","O","REG","REGN","RF","RSG","RMD","RHI","ROK","ROL","ROP","ROST","RCL","SPGI",
           "CRM","SBAC","SLB","STX","SEE","SRE","NOW","SHW","SPG","SWKS","SJM","SNA","SEDG","SO","LUV",
           "SWK","SBUX","STT","STLD","STE","SYK","SYF","SNPS","SYY","TMUS","TROW","TTWO","TPR","TRGP",
           "TGT","TEL","TDY","TFX","TER","TSLA","TXN","TXT","TMO","TJX","TSCO","TT","TDG","TRV","TRMB",
           "TFC","TYL","TSN","USB","UDR","ULTA","UNP","UAL","UPS","URI","UNH","UHS","VLO","VTR","VRSN",
           "VRSK","VZ","VRTX","VFC","VTRS","VICI","V","VMC","WAB","WBA","WMT","WBD","WM","WAT","WEC",
           "WFC","WELL","WST","WDC","WRK","WY","WHR","WMB","WTW","GWW","WYNN","XEL","XYL","YUM",
           "ZBRA","ZBH","ZION","ZTS"]

In [179]:
years = [2018,2019, 2020, 2021, 2022, 2023]
data = yf.download(tickers, start="2017-01-01", end="2023-01-01")
# Resample the data to get yearly prices
data.index = pd.to_datetime(data.index)
yearly_data = data["Adj Close"].resample("Y").last()
# Calculate the simple returns
simple_returns = yearly_data.pct_change()

# Create a dictionary with yearly returns for each ticker
returns_dict = {ticker: simple_returns[ticker].values for ticker in tickers}

# Add the yearly returns to your dataset as a new column (e.g., "target_variable")
df["target_variable"] = df.apply(lambda row: returns_dict[row["ticker"]][years.index(row["date"].year)], axis=1)

[*********************100%***********************]  503 of 503 completed

2 Failed downloads:
- BRK.B: No timezone found, symbol may be delisted
- BF.B: No data found for this date range, symbol may be delisted


In [180]:
# Calculate the simple returns
simple_returns = yearly_data.pct_change()

# Create a dictionary with yearly returns for each ticker
returns_dict = {ticker: simple_returns[ticker].values for ticker in tickers}

In [181]:
print(yearly_data)

                     A        AAL         AAP        AAPL        ABBV  \
Date                                                                    
2017-12-31   64.383583  50.711876   92.559265   40.168858   75.257759   
2018-12-31   65.455803  31.599049  146.444351   38.003681   74.537949   
2019-12-31   83.482079  28.574404  149.182877   71.810936   75.632294   
2020-12-31  116.700356  15.770000  147.844635  130.916138   96.596245   
2021-12-31  158.140182  17.959999  228.681335  176.276230  127.918579   
2022-12-31  149.406601  12.720000  145.212418  129.731918  158.628891   

                   ABC         ABT       ACGL         ACN        ADBE  ...  \
Date                                                                   ...   
2017-12-31   84.745964   52.067600  30.256666  141.294235  175.240005  ...   
2018-12-31   69.542999   67.199402  26.719999  132.493423  226.240005  ...   
2019-12-31   81.004173   82.037163  42.889999  200.348083  329.809998  ...   
2020-12-31   94.801720  1

In [182]:
sr = simple_returns.transpose()
sr = sr.drop(columns=['2017-12-31'])

In [183]:
sr.head()

Date,2018-12-31,2019-12-31,2020-12-31,2021-12-31,2022-12-31
A,0.016654,0.275396,0.397909,0.355096,-0.055227
AAL,-0.376891,-0.095719,-0.448107,0.138871,-0.291759
AAP,0.582168,0.018700,-0.008970,0.546768,-0.365001
AAPL,-0.053902,0.889578,0.823067,0.346482,-0.264042
ABBV,-0.009565,0.014682,0.277183,0.324260,0.240077


In [184]:
dumbframe = pd.DataFrame(columns = ["index","target_variable","Data"])
i = 0
for key in sr.keys():

  newframe = sr[key]
  newframe = newframe.to_frame()
  # print(newframe)
  newframe["Data"] = key
  newframe.reset_index(inplace=True)
  newframe.rename(columns={key:"target_variable"}, inplace=True)
  # print(newframe)
  # if i == 0:
  #   print("lolz")
  #   print(dumbframe)
  #   print(newframe)
  #   dumbframe = newframe
  #   i = 1
  # else:
  dumbframe = pd.concat([dumbframe,newframe])
  print(dumbframe)
  

    index  target_variable       Data
0       A         0.016654 2018-12-31
1     AAL        -0.376891 2018-12-31
2     AAP         0.582168 2018-12-31
3    AAPL        -0.053902 2018-12-31
4    ABBV        -0.009565 2018-12-31
..    ...              ...        ...
498   YUM         0.146064 2018-12-31
499   ZBH        -0.133042 2018-12-31
500  ZBRA         0.534008 2018-12-31
501  ZION        -0.182544 2018-12-31
502   ZTS         0.194473 2018-12-31

[503 rows x 3 columns]
    index  target_variable       Data
0       A         0.016654 2018-12-31
1     AAL        -0.376891 2018-12-31
2     AAP         0.582168 2018-12-31
3    AAPL        -0.053902 2018-12-31
4    ABBV        -0.009565 2018-12-31
..    ...              ...        ...
498   YUM         0.114145 2019-12-31
499   ZBH         0.453744 2019-12-31
500  ZBRA         0.604220 2019-12-31
501  ZION         0.309978 2019-12-31
502   ZTS         0.557090 2019-12-31

[1006 rows x 3 columns]
    index  target_variable       Data
0

In [185]:
type(sr["2018-12-31"])

pandas.core.series.Series

In [186]:
dumbframe.rename(columns={"Data":"date"}, inplace=True)

In [187]:
dumbframe.rename(columns={"index":"ticker"}, inplace=True)

In [188]:
dumbframe

,ticker,target_variable,date
0,A,0.016654,2018-12-31
1,AAL,-0.376891,2018-12-31
2,AAP,0.582168,2018-12-31
3,AAPL,-0.053902,2018-12-31
4,ABBV,-0.009565,2018-12-31
...,...,...,...
498,YUM,-0.059903,2022-12-31
499,ZBH,0.042350,2022-12-31
500,ZBRA,-0.569204,2022-12-31
501,ZION,-0.200226,2022-12-31


In [189]:
superframe = df.merge(dumbframe,on=["ticker","date"])

In [190]:
superframe.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1551 entries, 0 to 1550
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   ticker                1551 non-null   object        
 1   date                  1551 non-null   datetime64[ns]
 2   asset_debt_coverage   1535 non-null   float64       
 3   net_income            1511 non-null   float64       
 4   return_on_equity      1510 non-null   float64       
 5   return_on_assets      1511 non-null   float64       
 6   current_assets        1006 non-null   float64       
 7   current_liabilities   1363 non-null   float64       
 8   current_ratio         1005 non-null   float64       
 9   total_asset           1551 non-null   float64       
 10  total_liabilities     1551 non-null   float64       
 11  debt_to_equity_ratio  1550 non-null   float64       
 12  debt_to_assets_ratio  1551 non-null   float64       
 13  target_variable_x 

In [191]:
pd.unique(superframe["ticker"])


array(['MMM', 'AOS', 'ABT', 'ABBV', 'ATVI', 'ADM', 'AAP', 'AES', 'AFL',
       'AKAM', 'ALK', 'ALB', 'ARE', 'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL',
       'GOOG', 'MO', 'AMZN', 'AMD', 'AEE', 'AAL', 'AEP', 'AXP', 'AIG',
       'AMT', 'AWK', 'AMP', 'AME', 'AMGN', 'APH', 'ANSS', 'AON', 'APA',
       'APTV', 'ACGL', 'ANET', 'AJG', 'AIZ', 'T', 'AVB', 'AVY', 'AXON',
       'BKR', 'BALL', 'BAC', 'BAX', 'WRB', 'BIO', 'BIIB', 'BLK', 'BK',
       'BA', 'BKNG', 'BWA', 'BXP', 'BSX', 'BMY', 'BRO', 'BG', 'CHRW',
       'CDNS', 'CZR', 'CPT', 'COF', 'CARR', 'CAT', 'CBOE', 'CBRE', 'CDW',
       'CE', 'CNC', 'CNP', 'CDAY', 'CF', 'CRL', 'SCHW', 'CHTR', 'CVX',
       'CMG', 'CB', 'CHD', 'CI', 'CINF', 'C', 'CFG', 'CME', 'CMS', 'KO',
       'CTSH', 'CL', 'CMCSA', 'CMA', 'COP', 'ED', 'CEG', 'GLW', 'CTVA',
       'CSGP', 'CTRA', 'CCI', 'CSX', 'CMI', 'CVS', 'DHR', 'DVA', 'DAL',
       'XRAY', 'DVN', 'DXCM', 'FANG', 'DLR', 'DFS', 'DISH', 'D', 'DPZ',
       'DOV', 'DOW', 'DTE', 'DUK', 'DD', 'EMN', 'ETN', 'EBAY', 

In [192]:
lista_szmista = ['MMM', 'AOS', 'ABT', 'ABBV', 'ATVI', 'ADM', 'AAP', 'AES', 'AFL',
       'AKAM', 'ALK', 'ALB', 'ARE', 'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL',
       'GOOG', 'MO', 'AMZN', 'AMD', 'AEE', 'AAL', 'AEP', 'AXP', 'AIG',
       'AMT', 'AWK', 'AMP', 'AME', 'AMGN', 'APH', 'ANSS', 'AON', 'APA',
       'APTV', 'ACGL', 'ANET', 'AJG', 'AIZ', 'T', 'AVB', 'AVY', 'AXON',
       'BKR', 'BALL', 'BAC', 'BAX', 'WRB', 'BIO', 'BIIB', 'BLK', 'BK',
       'BA', 'BKNG', 'BWA', 'BXP', 'BSX', 'BMY', 'BRO', 'BG', 'CHRW',
       'CDNS', 'CZR', 'CPT', 'COF', 'CARR', 'CAT', 'CBOE', 'CBRE', 'CDW',
       'CE', 'CNC', 'CNP', 'CDAY', 'CF', 'CRL', 'SCHW', 'CHTR', 'CVX',
       'CMG', 'CB', 'CHD', 'CI', 'CINF', 'C', 'CFG', 'CME', 'CMS', 'KO',
       'CTSH', 'CL', 'CMCSA', 'CMA', 'COP', 'ED', 'CEG', 'GLW', 'CTVA',
       'CSGP', 'CTRA', 'CCI', 'CSX', 'CMI', 'CVS', 'DHR', 'DVA', 'DAL',
       'XRAY', 'DVN', 'DXCM', 'FANG', 'DLR', 'DFS', 'DISH', 'D', 'DPZ',
       'DOV', 'DOW', 'DTE', 'DUK', 'DD', 'EMN', 'ETN', 'EBAY', 'ECL',
       'EIX', 'EW', 'ELV', 'LLY', 'ENPH', 'ETR', 'EOG', 'EPAM', 'EQT',
       'EFX', 'EQIX', 'EQR', 'ESS', 'ETSY', 'RE', 'EVRG', 'ES', 'EXC',
       'EXPE', 'EXPD', 'EXR', 'XOM', 'FAST', 'FRT', 'FITB', 'FSLR', 'FE',
       'FIS', 'FISV', 'FLT', 'FMC', 'F', 'FTNT', 'FTV', 'FCX', 'GRMN',
       'IT', 'GEHC', 'GNRC', 'GD', 'GE', 'GM', 'GPC', 'GILD', 'GL', 'GPN',
       'GS', 'HAL', 'HIG', 'HAS', 'HCA', 'PEAK', 'HSIC', 'HSY', 'HES',
       'HLT', 'HON', 'HST', 'HWM', 'HUM', 'HBAN', 'HII', 'IBM', 'IEX',
       'IDXX', 'ITW', 'ILMN', 'INCY', 'IR', 'PODD', 'INTC', 'ICE', 'IFF',
       'IP', 'IPG', 'ISRG', 'IVZ', 'INVH', 'IQV', 'IRM', 'JBHT', 'JNJ',
       'JPM', 'JNPR', 'K', 'KDP', 'KEY', 'KMB', 'KIM', 'KMI', 'KHC',
       'LHX', 'LH', 'LVS', 'LDOS', 'LNC', 'LIN', 'LYV', 'LKQ', 'LMT', 'L',
       'LYB', 'MTB', 'MRO', 'MPC', 'MKTX', 'MAR', 'MMC', 'MLM', 'MAS',
       'MA', 'MTCH', 'MCD', 'MRK', 'META', 'MET', 'MTD', 'MGM', 'MAA',
       'MRNA', 'MHK', 'MOH', 'TAP', 'MDLZ', 'MPWR', 'MNST', 'MCO', 'MS',
       'MOS', 'MSI', 'MSCI', 'NDAQ', 'NFLX', 'NWL', 'NEM', 'NEE', 'NI',
       'NSC', 'NTRS', 'NOC', 'NCLH', 'NRG', 'NUE', 'NVR', 'NXPI', 'ORLY',
       'OXY', 'ODFL', 'OMC', 'ON', 'OKE', 'OGN', 'OTIS', 'PCAR', 'PKG',
       'PARA', 'PAYC', 'PYPL', 'PNR', 'PEP', 'PKI', 'PFE', 'PCG', 'PM',
       'PSX', 'PNW', 'PXD', 'PNC', 'POOL', 'PPG', 'PPL', 'PFG', 'PGR',
       'PLD', 'PRU', 'PEG', 'PSA', 'PHM', 'PWR', 'DGX', 'RTX', 'O', 'REG',
       'REGN', 'RF', 'RSG', 'RHI', 'ROL', 'ROP', 'RCL', 'SPGI', 'SBAC',
       'SLB', 'SEE', 'SRE', 'NOW', 'SHW', 'SPG', 'SNA', 'SEDG', 'SO',
       'LUV', 'SWK', 'STT', 'STLD', 'SYK', 'SYF', 'TMUS', 'TROW', 'TRGP',
       'TDY', 'TFX', 'TER', 'TSLA', 'TXN', 'TXT', 'TMO', 'TSCO', 'TT',
       'TRV', 'TRMB', 'TFC', 'TYL', 'USB', 'UDR', 'UNP', 'UAL', 'UPS',
       'URI', 'UNH', 'UHS', 'VLO', 'VTR', 'VRSN', 'VRSK', 'VZ', 'VRTX',
       'VTRS', 'VICI', 'VMC', 'WAB', 'WBD', 'WM', 'WAT', 'WEC', 'WFC',
       'WELL', 'WST', 'WY', 'WHR', 'WMB', 'WTW', 'GWW', 'WYNN', 'XEL',
       'XYL', 'YUM', 'ZBRA', 'ZBH', 'ZION', 'ZTS']

print(500-len(lista_szmista))

112


In [193]:
df = superframe

In [197]:
# Calculate the simple returns
simple_returns = yearly_data.pct_change()

# Create a dictionary with yearly returns for each ticker
returns_dict = {ticker: simple_returns[ticker].values for ticker in tickers}

# Add the yearly returns to your dataset as a new column (e.g., "target_variable")
df["target_variable"] = df.apply(lambda row: returns_dict[row["ticker"]][years.index(row["date"].year)], axis=1)

In [195]:
# years = [2019, 2020, 2021, 2022]
# data = yf.download(tickers, start="2018-01-01", end="2023-01-01", interval="1y")

# # Calculate the simple returns
# simple_returns = data["Adj Close"].pct_change().dropna()

# # Create a dictionary with yearly returns for each ticker
# returns_dict = {ticker: simple_returns[ticker].values for ticker in tickers}



In [199]:
df

,ticker,date,asset_debt_coverage,net_income,return_on_equity,return_on_assets,current_assets,current_liabilities,current_ratio,total_asset,total_liabilities,debt_to_equity_ratio,debt_to_assets_ratio,target_variable_x,target_variable_y,target_variable
date,,,,,,,,,,,,,,,,
2019-12-31,MMM,2019-12-31,2.172130,5.027400e+10,4.686976e+09,1.125731,1.127800e+10,9.222000e+09,1.222945,4.465900e+10,3.453300e+10,3.410330,0.773260,-0.168991,-0.042908,-0.168991
2020-12-31,MMM,2020-12-31,2.485119,5.142200e+10,3.754085e+09,1.086136,1.357800e+10,7.948000e+09,1.708354,4.734400e+10,3.441300e+10,2.661279,0.726871,-0.042908,0.027652,-0.042908
2021-12-31,MMM,2021-12-31,2.670600,5.257100e+10,3.282977e+09,1.116821,1.420900e+10,9.035000e+09,1.572662,4.707200e+10,3.195500e+10,2.113845,0.678854,0.027652,0.048506,0.027652
2022-12-31,MMM,2022-12-31,2.867593,5.462300e+10,3.491261e+09,1.175826,1.355900e+10,9.523000e+09,1.423816,4.645500e+10,3.168500e+10,2.145227,0.682058,0.048506,-0.296264,0.048506
2022-12-31,AOS,2022-12-31,9.082311,2.967400e+09,3.235789e+08,0.890496,1.488800e+09,9.342000e+08,1.593663,3.332300e+09,1.584600e+09,0.906677,0.475527,0.592783,-0.320743,0.592783
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-31,ZION,2022-12-31,137.549923,8.923000e+09,2.711075e+08,0.099648,NaN,NaN,NaN,8.954500e+10,8.465200e+10,19.010106,0.945357,0.491063,-0.200226,0.491063
2019-12-31,ZTS,2019-12-31,1.746332,5.153000e+09,9.550390e+08,0.446340,4.428000e+09,1.806000e+09,2.451827,1.154500e+10,8.837000e+09,3.263294,0.765440,0.194473,0.557090,0.194473
2020-12-31,ZTS,2020-12-31,1.848547,6.389000e+09,8.507782e+08,0.469469,6.243000e+09,2.170000e+09,2.876959,1.360900e+10,9.836000e+09,2.609711,0.722757,0.557090,0.257636,0.557090


In [198]:
df = df.set_index(df["date"])
df = df.drop(columns=["index"])

KeyError: ignored

In [200]:
df.head()

,ticker,date,asset_debt_coverage,net_income,return_on_equity,return_on_assets,current_assets,current_liabilities,current_ratio,total_asset,total_liabilities,debt_to_equity_ratio,debt_to_assets_ratio,target_variable_x,target_variable_y,target_variable
date,,,,,,,,,,,,,,,,
2019-12-31,MMM,2019-12-31,2.172130,5.027400e+10,4.686976e+09,1.125731,1.127800e+10,9.222000e+09,1.222945,4.465900e+10,3.453300e+10,3.410330,0.773260,-0.168991,-0.042908,-0.168991
2020-12-31,MMM,2020-12-31,2.485119,5.142200e+10,3.754085e+09,1.086136,1.357800e+10,7.948000e+09,1.708354,4.734400e+10,3.441300e+10,2.661279,0.726871,-0.042908,0.027652,-0.042908
2021-12-31,MMM,2021-12-31,2.670600,5.257100e+10,3.282977e+09,1.116821,1.420900e+10,9.035000e+09,1.572662,4.707200e+10,3.195500e+10,2.113845,0.678854,0.027652,0.048506,0.027652
2022-12-31,MMM,2022-12-31,2.867593,5.462300e+10,3.491261e+09,1.175826,1.355900e+10,9.523000e+09,1.423816,4.645500e+10,3.168500e+10,2.145227,0.682058,0.048506,-0.296264,0.048506
2022-12-31,AOS,2022-12-31,9.082311,2.967400e+09,3.235789e+08,0.890496,1.488800e+09,9.342000e+08,1.593663,3.332300e+09,1.584600e+09,0.906677,0.475527,0.592783,-0.320743,0.592783


# Neural Network

In [201]:
# Preprocessing: Fill missing values with the mean of each column
imputer = SimpleImputer(missing_values=np.nan, strategy="mean")
imputed_df = pd.DataFrame(imputer.fit_transform(df.drop(["ticker", "date"], axis=1)), columns=df.drop(["ticker", "date"], axis=1).columns)
df[df.drop(["ticker", "date"], axis=1).columns] = imputed_df


In [202]:
 # Normalize the features
scaler = MinMaxScaler()
df_normalized = scaler.fit_transform(df.drop(["ticker", "date", "target_variable"], axis=1))

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_data.py:473: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_data.py:474: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)


#NAPRAWIĆ PODZIAŁ

In [205]:
from sklearn.model_selection import TimeSeriesSplit

# Create a TimeSeriesSplit object
tscv = TimeSeriesSplit(n_splits=5)

# Split the data into training and testing sets
for train_index, test_index in tscv.split(df):
  print(train_index)
  print(test_index)
  print(tscv.split(df))
  X_train, X_test = df[train_index], df[test_index]
  y_train, y_test = df["target_variable"][train_index], df["target_variable"][test_index]

[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 235 236 237 238 239 240 241 242 243 244 245 24

KeyError: ignored

In [ ]:
# # Split data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(df_normalized, df["target_variable"], test_size=0.2, random_state=42)

In [ ]:
# Define the neural network model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation="relu", input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(32, activation="relu"),
    tf.keras.layers.Dense(1)
])

In [ ]:
#  Compile the model
model.compile(optimizer="adam", loss="mean_squared_error", metrics=["mean_absolute_error"])

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test), verbose=0)


In [ ]:
# Predict the target variable for all companies
predictions = model.predict(df_normalized).flatten()

# Add predictions to the dataframe
df["predicted_target"] = predictions

# Select the top 10 companies based on the predicted target variable
best_companies = df.nlargest(10, "predicted_target")

# Print the top 10 companies
print(best_companies[["ticker", "predicted_target"]])